In [ ]:
# !pip install inference-sdk

In [ ]:
# !pip install webcolors

In [57]:
import webcolors as w
import cv2
import matplotlib.pyplot as plt
from scipy.stats import mode
import numpy as np


In [65]:
def get_letter(ara_letter)-> str:
  """
     A function named 'get_letter' that translates English letters into their corresponding Arabic letters.

     Parameters:
        - ara_letter {string}: The English letter to be translated into Arabic.

     Returns:
        - A string representing the English equivalent of the provided Arabic letter.
  """
  eng_letter=""
  classes=['aain', 'ain', 'alf' ,'baa' ,'dal', 'faa' ,'geem' ,'haa', 'kaf' ,'lam' ,'meem' ,'non' ,'raa', 'sad','sen' ,'taa' ,'waaw' ,'yaa' ,'zen']
  if ara_letter=='aain':
    eng_letter="غ"
  if ara_letter=='ain':
    eng_letter="ع"
  if ara_letter=='alf':
    eng_letter="أ"
  if ara_letter=='baa':
    eng_letter="ب"
  if ara_letter=='dal':
    eng_letter="د"
  if ara_letter=='faa':
    eng_letter="ف"
  if ara_letter=="geem":
    eng_letter="ج"
  if ara_letter=='haa':
    eng_letter="ه"
  if ara_letter=='kaf':
    eng_letter="ق"
  if ara_letter=='lam':
    eng_letter="ل"
  if ara_letter=='meem':
    eng_letter="م"
  if ara_letter=='non':
    eng_letter="ن"
  if ara_letter=='raa':
    eng_letter="ر"
  if ara_letter=='sad':
    eng_letter="ص"
  if ara_letter=='sen':
    eng_letter="س"
  if ara_letter=='taa':
    eng_letter="ط"
  if ara_letter=='waaw':
    eng_letter="و"
  if ara_letter=='yaa':
    eng_letter="ي"
  if ara_letter=='zen':
    eng_letter="ز"
  return eng_letter

In [64]:
def is_int(value)-> bool:
    """
      A function named 'is_int' that checks whether a given value can be converted to an integer.

      Parameters:
      - value: The input value to be checked for its convertibility to an integer.

      Returns:
      - True if the input value can be successfully converted to an integer.
      - False if the input value cannot be converted to an integer (raises a ValueError)
    """
    try:
        int(value)
        return True
    except ValueError:
        return False


In [67]:
def get_c(L,N) -> str:
  """
    A function named 'get_c' that determines the city based on the given input parameters, where:

    Parameters:
    - L: The input string representing the first letter(s) of a city or region.
    - N: The input string representing a numbers of the plate.

    Returns:
    - A string representing the determined city based on the provided input parameters.
  """
  L = L.replace(" ", "")
  N = N.replace(" ", "")
  hroof=['س','ر','د','م','ب','ل','ع','ق','ف','و','ن','ي','ه']
  hroof2=['جه','جب','صأ','صق','صو','طس','طص','طع','طد' ,'طأ','طج','طر']
  cities=['الإسكندرية','الشرقية','الدقهلية','المنوفية','البحيرة','كفر الشيخ','الغربية','القليوبية','الفيوم','بني سويف','المنيا','أسيوط','سوهاج']
  cities2=['مطروح','الوادي الجديد','قنا','الأقصر','أسوان','السويس','الإسماعيلية','بورسعيد','دمياط',	'شمال سيناء','جنوب سيناء','البحر الأحمر']
  c=""
  if L[0] in hroof and len(N)==4 and len(L)==3:
    c=cities[hroof.index(L[0])]
  elif L[:2] in hroof2 and len(N)==4 and len(L)==3:
   c=cities2[hroof2.index(L[:2])]
  else:
    if len(L)==2 and len(N)==4:
      c="الجيزة"
    else:
      c="القاهرة"
  return c

# get_c(" ه ب ج",'6214')


In [72]:
def get_car_type(color):
  car_type=""
  if 'blue' in color:
    if 'dark' in color:
      car_type="شرطة"
    else:
      car_type="ملاكي"

  elif 'orange' in color:
    car_type="أجرة"

  elif 'red' in color:
    car_type="النقل أوالجرارات"

  elif 'brown' in color:
    car_type="تجارية"

  elif 'yellow' in color:
    car_type="الجمارك"

  elif 'green' in color:
    car_type="هيئات سياسية"

  else:
    car_type="لم نستطع تحديد نوع الالسيارة"
  return car_type

In [60]:
def cloest_color(rgb):
  d={}
  for c_hex,c_name in w.CSS3_HEX_TO_NAMES.items():
    r,g,b=w.hex_to_rgb(c_hex)
    d[sum([(r-rgb[0])**2,(g-rgb[1])**2,(b-rgb[2])**2])]=c_name
  return d[min(d.keys())]


In [61]:
def getColor(im_path):
  img=cv2.cvtColor(cv2.imread(im_path),cv2.COLOR_BGR2RGB)
  processed_image = cv2.resize(img, (700, 480))
  Color = processed_image[30:80,320:400]
  mean_pixel_values = np.mean(Color, axis=(0, 1))
  mean_pixel_values=mean_pixel_values.astype('i8')
  color=(mean_pixel_values[0],mean_pixel_values[1],mean_pixel_values[2])
  cloest_color(color)
  c=""
  try :
    c=w.rgb_to_name(color)
  except ValueError:
    c=cloest_color(color)
  return c

# getColor("/content/R (1).jpeg")

In [ ]:
from inference_sdk import InferenceHTTPClient
def get_Letters_and_nums(im_path) -> dict:
  """
    A function named 'get_Letters_and_nums' that processes an image to detect letters and numbers on license plates using a pre-trained model.

    Parameters:
    - im_path: The path to the input image file.

    Returns:
    - A dictionary containing the detected letters, numbers, and the determined city based on the license plate information.
  """
  CLIENT = InferenceHTTPClient(api_url="https://detect.roboflow.com",api_key="JzjabeiSHDBEgPuCkxiZ")
  result = CLIENT.infer(im_path, model_id="plate-detect-oydq2/1")
  sorted_detections = sorted(result['predictions'], key=lambda x: x['x'])
  res2={"letters":"",'numbers':""}
  for detection in sorted_detections:
    x=detection['class']
    isInt=is_int(x)
    if isInt==False:
      x2=get_letter(x)
      res2['letters']+=x2+' '
    elif isInt:
      res2['numbers']+=str(x)+' '
  res2['letters']=res2['letters'][::-1]
  if res2['letters']=='':
      res2['letters']="لا توجد حروف على هذه السيارة"
      res2['city']="لا يمكن معرفة المدينة لان السيارة تابعة لهيئات سياسية"
  else:
    res2['city']=get_c(res2['letters'],res2['numbers'])

  car_type=get_car_type(getColor(im_path))
  res2["car type"]=car_type

  return res2

# get_Letters_and_nums("/content/OIP (2).jpeg")



